In [22]:
import os
import shutil
import scipy.io
import pickle
import numpy as np
from sklearn import preprocessing
from datetime import datetime, date, time
import pandas as pd
import torch
import torch.nn as nn
import logging
from torchviz import make_dot
from torchsummary import summary

In [4]:
device = "mps" if torch.backends.mps.is_available() else 'cpu'

# Extract data from Zip

In [2]:
data_path = './Dataset/'
if not os.path.exists(data_path):
    os.makedirs(data_path)

filename = '/SEED-V.zip'
shutil.unpack_archive(filename, data_path)

ReadError: /SEED-V.zip is not a zip file

# Load data and label from npz

In [5]:
data_npz = np.load('/Users/calebjonesshibu/Documents/UofA/Classes/Sem3/Thesis/deep-learning-emotion-recognition/Dataset/SEED-V/EEG_DE_features/1_123.npz')
print(data_npz.files)

['data', 'label']


In [6]:
data = pickle.loads(data_npz['data'])
label = pickle.loads(data_npz['label'])

print(data.keys())
print(label.keys())

# As we can see, there are 45 keys in both 'data' and 'label'.
# Each participant took part in our experiments for 3 sessions, and he/she watched 15 movie clips (i.e. 15 trials) during each session.
# Therefore, we could extract 3 * 15 = 45 DE feature matrices.

# The key indexes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14] belong to Session 1.
# The key indexes [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29] belong to Session 2.
# The key indexes [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44] belong to Session 3.

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44])


In [7]:
# We will print the emotion labels for each trial.
label_dict = {0:'Disgust', 1:'Fear', 2:'Sad', 3:'Neutral', 4:'Happy'}
for i in range(45):
    print('Session {} -- Trial {} -- EmotionLabel : {}'.format(i//15+1, i%15+1, label_dict[label[i][0]]))

Session 1 -- Trial 1 -- EmotionLabel : Happy
Session 1 -- Trial 2 -- EmotionLabel : Fear
Session 1 -- Trial 3 -- EmotionLabel : Neutral
Session 1 -- Trial 4 -- EmotionLabel : Sad
Session 1 -- Trial 5 -- EmotionLabel : Disgust
Session 1 -- Trial 6 -- EmotionLabel : Happy
Session 1 -- Trial 7 -- EmotionLabel : Fear
Session 1 -- Trial 8 -- EmotionLabel : Neutral
Session 1 -- Trial 9 -- EmotionLabel : Sad
Session 1 -- Trial 10 -- EmotionLabel : Disgust
Session 1 -- Trial 11 -- EmotionLabel : Happy
Session 1 -- Trial 12 -- EmotionLabel : Fear
Session 1 -- Trial 13 -- EmotionLabel : Neutral
Session 1 -- Trial 14 -- EmotionLabel : Sad
Session 1 -- Trial 15 -- EmotionLabel : Disgust
Session 2 -- Trial 1 -- EmotionLabel : Sad
Session 2 -- Trial 2 -- EmotionLabel : Fear
Session 2 -- Trial 3 -- EmotionLabel : Neutral
Session 2 -- Trial 4 -- EmotionLabel : Disgust
Session 2 -- Trial 5 -- EmotionLabel : Happy
Session 2 -- Trial 6 -- EmotionLabel : Happy
Session 2 -- Trial 7 -- EmotionLabel : Disgus

In [8]:
len(label), len(data)

(45, 45)

In [9]:
data[1].shape

(24, 310)

# Load data as train test 

In [10]:
def loading_cv_data(eeg_dir, eye_dir, file_name, cv_number):
    eeg_data_pickle = np.load( os.path.join(eeg_dir, file_name))
    eye_data_pickle = np.load( os.path.join(eye_dir, file_name))
    eeg_data = pickle.loads(eeg_data_pickle['data'])
    eye_data = pickle.loads(eye_data_pickle['data'])
    label = pickle.loads(eeg_data_pickle['label'])
    list_1 = [0,1,2,3,4,15,16,17,18,19,30,31,32,33,34]
    list_2 = [5,6,7,8,9,20,21,22,23,24,35,36,37,38,39]
    list_3 = [10,11,12,13,14,25,26,27,28,29,40,41,42,43,44]
    if cv_number == 1:
        print('#1 as test, preparing data')
        train_list = list_2 + list_3
        test_list = list_1
    elif cv_number == 2:
        print('#2 as test, preparing data')
        train_list = list_1 + list_3
        test_list = list_2
    else:
        print('#3 as test, preparing data')
        train_list = list_1 + list_2
        test_list = list_3

    train_eeg = []
    test_eeg = []
    train_label = []
    for train_id in range(len(train_list)):
        train_eeg_tmp = eeg_data[train_list[train_id]]
        train_eye_tmp = eye_data[train_list[train_id]]
        train_label_tmp = label[train_list[train_id]]
        if train_id == 0:
            train_eeg = train_eeg_tmp
            train_eye = train_eye_tmp
            train_label = train_label_tmp
        else:
            train_eeg = np.vstack((train_eeg, train_eeg_tmp))
            train_eye = np.vstack((train_eye, train_eye_tmp))
            train_label = np.hstack((train_label, train_label_tmp))
    assert train_eeg.shape[0] == train_eye.shape[0]
    assert train_eeg.shape[0] == train_label.shape[0]

    test_eeg = []
    test_eye = []
    test_label = []
    for test_id in range(len(test_list)):
        test_eeg_tmp = eeg_data[test_list[test_id]]
        test_eye_tmp = eye_data[test_list[test_id]]
        test_label_tmp = label[test_list[test_id]]
        if test_id == 0:
            test_eeg = test_eeg_tmp
            test_eye = test_eye_tmp
            test_label = test_label_tmp
        else:
            test_eeg = np.vstack((test_eeg, test_eeg_tmp))
            test_eye = np.vstack((test_eye, test_eye_tmp))
            test_label = np.hstack((test_label, test_label_tmp))
    assert test_eeg.shape[0] == test_eye.shape[0]
    assert test_eeg.shape[0] == test_label.shape[0]

    train_all = np.hstack((train_eeg, train_eye, train_label.reshape([-1,1])))
    test_all = np.hstack((test_eeg, test_eye, test_label.reshape([-1,1])))
    return train_all, test_all

In [11]:
eeg_dir = '/Users/calebjonesshibu/Documents/UofA/Classes/Sem3/Thesis/deep-learning-emotion-recognition/Dataset/SEED-V/EEG_DE_features/'
eye_dir = '/Users/calebjonesshibu/Documents/UofA/Classes/Sem3/Thesis/deep-learning-emotion-recognition/Dataset/SEED-V/Eye_movement_features/'
file_list = os.listdir(eeg_dir)
file_list.sort()

# Define Parts of Neural Network

In [12]:
def cca_metric_derivative(H1, H2):
    r1 = 1e-3
    r2 = 1e-3
    eps = 1e-9
    # transform the matrix: to be consistent with the original paper
    H1 = H1.T
    H2 = H2.T
    # o1 and o2 are feature dimensions
    # m is sample number
    o1 = o2 = H1.shape[0]
    m = H1.shape[1]

    # calculate parameters
    H1bar = H1 - H1.mean(axis=1).reshape([-1,1])
    H2bar = H2 - H2.mean(axis=1).reshape([-1,1])

    SigmaHat12 = (1.0 / (m - 1)) * np.matmul(H1bar, H2bar.T)
    SigmaHat11 = (1.0 / (m - 1)) * np.matmul(H1bar, H1bar.T) + r1 * np.eye(o1)
    SigmaHat22 = (1.0 / (m - 1)) * np.matmul(H2bar, H2bar.T) + r2 * np.eye(o2)

    # eigenvalue and eigenvector decomposition
    [D1, V1] = np.linalg.eigh(SigmaHat11)
    [D2, V2] = np.linalg.eigh(SigmaHat22)

    # remove eighvalues and eigenvectors smaller than 0
    posInd1 = np.where(D1 > 0)[0]
    D1 = D1[posInd1]
    V1 = V1[:, posInd1]

    posInd2 = np.where(D2 > 0)[0]
    D2 = D2[posInd2]
    V2 = V2[:, posInd2]

    # calculate matrxi T
    SigmaHat11RootInv = np.matmul(np.matmul(V1, np.diag(D1 ** -0.5)), V1.T)
    SigmaHat22RootInv = np.matmul(np.matmul(V2, np.diag(D2 ** -0.5)), V2.T)
    Tval = np.matmul(np.matmul(SigmaHat11RootInv,SigmaHat12), SigmaHat22RootInv)
    # By default, we will use all the singular values
    tmp = np.matmul(Tval.T, Tval)
    corr = np.sqrt(np.trace(tmp))
    cca_loss = -1 * corr

    # calculate the derivative of H1 and H2
    U_t, D_t, V_prime_t = np.linalg.svd(Tval)
    Delta12 = SigmaHat11RootInv @ U_t @ V_prime_t @ SigmaHat22RootInv
    Delta11 = SigmaHat11RootInv @ U_t @ np.diag(D_t) @ U_t.T @ SigmaHat11RootInv
    Delta22 = SigmaHat22RootInv @ U_t @ np.diag(D_t) @ U_t.T @ SigmaHat22RootInv
    Delta11 = -0.5 * Delta11
    Delta22 = -0.5 * Delta22

    DerivativeH1 = ( 1.0 / (m - 1)) * (2 * (Delta11 @ H1bar) + Delta12 @ H2bar)
    DerivativeH2 = ( 1.0 / (m - 1)) * (2 * (Delta22 @ H2bar) + Delta12 @ H1bar)

    return cca_loss, DerivativeH1.T, DerivativeH2.T
    
class AttentionFusion(nn.Module):
    def __init__(self, output_dim):
        super(AttentionFusion, self).__init__()
        self.output_dim = output_dim
        self.attention_weights = nn.Parameter(torch.randn(self.output_dim, requires_grad=True))
    def forward(self, x1, x2):
        # calculate weigths for all input samples
        row, _ = x1.shape
        fused_tensor = torch.empty_like(x1)
        alpha = []
        for i in range(row):
            tmp1 = torch.dot(x1[i,:], self.attention_weights)
            tmp2 = torch.dot(x2[i,:], self.attention_weights)
            alpha_1 = torch.exp(tmp1) / (torch.exp(tmp1) + torch.exp(tmp2))
            alpha_2 = 1 - alpha_1
            alpha.append((alpha_1.detach().cpu().numpy(), alpha_2.detach().cpu().numpy()))
            fused_tensor[i, :] = alpha_1 * x1[i,:] + alpha_2 * x2[i, :]
        return fused_tensor, alpha

class TransformLayers(nn.Module):
    def __init__(self, input_size, layer_sizes):
        super(TransformLayers, self).__init__()
        layers = []
        layer_sizes = [input_size] + layer_sizes
        for l_id in range(len(layer_sizes) - 1):
            if l_id == len(layer_sizes) - 2:
                layers.append(nn.Sequential(
                    #nn.BatchNorm1d(num_features=layer_sizes[l_id], affine=False),
                    nn.Linear(layer_sizes[l_id], layer_sizes[l_id+1]),
                    ))
            else:
                layers.append(nn.Sequential(
                    nn.Linear(layer_sizes[l_id], layer_sizes[l_id+1]),
                    nn.Sigmoid(),
                    #nn.BatchNorm1d(num_features=layer_sizes[l_id+1], affine=False),
                    ))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class DCCA_AM(nn.Module):
    def __init__(self, input_size1, input_size2, layer_sizes1, layer_sizes2, outdim_size, categories, device):
        super(DCCA_AM, self).__init__()
        self.outdim_size = outdim_size
        self.categories = categories
        # self.use_all_singular_values = use_all_singular_values
        self.device = device

        self.model1 = TransformLayers(input_size1, layer_sizes1).to(self.device)
        self.model2 = TransformLayers(input_size2, layer_sizes2).to(self.device)

        self.model1_parameters = self.model1.parameters()
        self.model2_parameters = self.model1.parameters()

        self.classification = nn.Linear(self.outdim_size, self.categories)

        self.attention_fusion = AttentionFusion(outdim_size)
    def forward(self, x1, x2):
        # forward process: returns negative of cca loss and predicted labels
        output1 = self.model1(x1)
        output2 = self.model2(x2)
        # cca_loss_val = self.loss(output1, output2)
        cca_loss, partial_h1, partial_h2 = cca_metric_derivative(output1.detach().cpu().numpy(), output2.detach().cpu().numpy())
        fused_tensor, alpha = self.attention_fusion(output1, output2)
        out = self.classification(fused_tensor)
        return out, cca_loss, output1, output2, partial_h1, partial_h2, fused_tensor.detach().cpu().data, alpha


In [13]:
batch_size = 50
emotion_categories = 5

epochs = 70
eeg_input_dim = 310
eye_input_dim = 33
output_dim = 12
learning_rate = 5 * 1e-4
batch_size = 50

In [24]:
cv =3
for f_id in file_list:
    type(f_id)
    logging.basicConfig(filename='./logs/cv3.log', level=logging.DEBUG)
    logging.debug('{}'.format(f_id))
    logging.debug('Task-Epoch-CCALoss-PredicLoss-PredicAcc')
    if f_id.endswith('.npz'):
        print(f_id)
        train_all, test_all = loading_cv_data(eeg_dir, eye_dir, f_id, cv)
        np.random.shuffle(train_all)
        np.random.shuffle(test_all)

        sample_num = train_all.shape[0]
        batch_number = sample_num // batch_size

        train_eeg = train_all[:, 0:310]
        train_eye = train_all[:, 310:343]
        train_label = train_all[:, -1]

        scaler = preprocessing.MinMaxScaler()
        train_eeg = scaler.fit_transform(train_eeg)
        train_eye = scaler.fit_transform(train_eye)
        test_eeg = test_all[:, 0:310]
        test_eye = test_all[:, 310:343]
        test_label = test_all[:, -1]

        test_eeg = scaler.fit_transform(test_eeg)
        test_eye = scaler.fit_transform(test_eye)

        train_eeg = torch.from_numpy(train_eeg).to(torch.float).to(device)
        train_eye = torch.from_numpy(train_eye).to(torch.float).to(device)
        test_eeg = torch.from_numpy(test_eeg).to(torch.float).to(device)
        test_eye = torch.from_numpy(test_eye).to(torch.float).to(device)
        train_label = torch.from_numpy(train_label).to(torch.long).to(device)
        test_label = torch.from_numpy(test_label).to(torch.long).to(device)

        for hyper_choose in range(100):

                best_test_res = {}
                best_test_res['acc'] = 0
                best_test_res['predict_proba'] = None
                best_test_res['fused_feature'] = None
                best_test_res['transformed_eeg'] = None
                best_test_res['transformed_eye'] = None
                best_test_res['alpha'] = None
                best_test_res['true_label'] = None
                best_test_res['layer_size'] = None
                # try 100 combinations of different hidden units
                layer_sizes = [np.random.randint(100,200), np.random.randint(20,50), output_dim]
                logging.info('{}-{}'.format(layer_sizes[0], layer_sizes[1]))
                print(layer_sizes)
                mymodel = DCCA_AM(eeg_input_dim, eye_input_dim, layer_sizes, layer_sizes, output_dim, emotion_categories, device).to(device)
                
                summary(mymodel, input_size = eeg_input_dim)                
                optimizer_classifier = torch.optim.RMSprop(mymodel.parameters(), lr=learning_rate)
                optimizer_model1 = torch.optim.RMSprop(iter(list(mymodel.parameters())[0:8]), lr=learning_rate/2)
                optimizer_model2 = torch.optim.RMSprop(iter(list(mymodel.parameters())[8:16]), lr=learning_rate/2)
                class_loss_func = nn.CrossEntropyLoss()
                for epoch in range(epochs):
                    mymodel.train()
                    best_acc = 0
                    total_classification_loss = 0
                    for b_id in range(batch_number+1):
                        if b_id == batch_number:
                            train_eeg_used = train_eeg[batch_size*batch_number:, :]
                            train_eye_used = train_eye[batch_size*batch_number: , :]
                            train_label_used = train_label[batch_size*batch_number:]
                        else:
                            train_eeg_used = train_eeg[b_id*batch_size:(b_id+1)*batch_size, :]
                            train_eye_used = train_eye[b_id*batch_size:(b_id+1)*batch_size, :]
                            train_label_used = train_label[b_id*batch_size:(b_id+1)*batch_size]

                        # predict_out, cca_loss, output1, output2, partial_h1, partial_h2, fused_tensor, transformed_1, transformed_2, alpha  = mymodel(train_eeg_used, train_eye_used)
                        predict_out, cca_loss, output1, output2, partial_h1, partial_h2, fused_tensor, alpha  = mymodel(train_eeg_used, train_eye_used)
                        predict_loss = class_loss_func(predict_out, train_label_used)

                        optimizer_model1.zero_grad()
                        optimizer_model2.zero_grad()
                        optimizer_classifier.zero_grad()

                        partial_h1 = torch.from_numpy(partial_h1).to(torch.float).to(device)
                        partial_h2 = torch.from_numpy(partial_h2).to(torch.float).to(device)

                        output1.backward(-0.1*partial_h1, retain_graph=True)
                        output2.backward(-0.1*partial_h2, retain_graph=True)
                        predict_loss.backward()

                        optimizer_model1.step()
                        optimizer_model2.step()
                        optimizer_classifier.step()
                    # for every epoch, evaluate the model on both train and test set
                    mymodel.eval()
                    predict_out_train, cca_loss_train, _, _, _, _, _, _  = mymodel(train_eeg, train_eye)
                    predict_loss_train = class_loss_func(predict_out_train, train_label)
                    accuracy_train = np.sum(np.argmax(predict_out_train.detach().cpu().numpy(), axis=1) == train_label.detach().cpu().numpy()) / predict_out_train.shape[0]

                    predict_out_test, cca_loss_test, output_1_test, output_2_test, _, _, fused_tensor_test, attention_weight_test  = mymodel(test_eeg, test_eye)
                    predict_loss_test = class_loss_func(predict_out_test, test_label)
                    accuracy_test = np.sum(np.argmax(predict_out_test.detach().cpu().numpy(), axis=1) == test_label.detach().cpu().numpy()) / predict_out_test.shape[0]

                    if accuracy_test > best_test_res['acc']:
                        best_test_res['acc'] = accuracy_test
                        best_test_res['layer_size'] = layer_sizes
                        best_test_res['predict_proba'] = predict_out_test.detach().cpu().data
                        best_test_res['fused_feature'] = fused_tensor_test
                        best_test_res['transformed_eeg'] = output_1_test.detach().cpu().data
                        best_test_res['transformed_eye'] = output_2_test.detach().cpu().data
                        best_test_res['alpha'] = attention_weight_test
                        best_test_res['true_label'] = test_label.detach().cpu().data

                    print('Epoch: {} -- Train CCA loss is: {} -- Train loss: {} -- Train accuracy: {}'.format(epoch, cca_loss_train, predict_loss_train.data, accuracy_train))
                    print('Epoch: {} -- Test CCA loss is: {} -- Test loss: {} -- Test accuracy: {}'.format(epoch, cca_loss_test, predict_loss_test.data, accuracy_test))
                    print('\n')
                    logging.info('\tTrain\t{}\t{}\t{}\t{}'.format(epoch, cca_loss_train, predict_loss_train.data, accuracy_train))
                    logging.info('\tTest\t{}\t{}\t{}\t{}'.format(epoch, cca_loss_test, predict_loss_test.data, accuracy_test))

                pickle.dump(best_test_res, open( os.path.join('./', f_id[:-8]+'_'+str(hyper_choose)), 'wb'  ))



10_123.npz
#3 as test, preparing data
[161, 34, 12]
Layer (type:depth-idx)                   Param #
├─TransformLayers: 1-1                   --
|    └─ModuleList: 2-1                   --
|    |    └─Sequential: 3-1              50,071
|    |    └─Sequential: 3-2              5,508
|    |    └─Sequential: 3-3              420
├─TransformLayers: 1-2                   --
|    └─ModuleList: 2-2                   --
|    |    └─Sequential: 3-4              5,474
|    |    └─Sequential: 3-5              5,508
|    |    └─Sequential: 3-6              420
├─Linear: 1-3                            65
├─AttentionFusion: 1-4                   12
Total params: 67,478
Trainable params: 67,478
Non-trainable params: 0


KeyboardInterrupt: 

In [12]:
train_eye.shape, train_eeg.shape

((1199, 33), (1199, 310))